In [17]:
from dolfin import *
from mshr import *
import math

l = 1.
tol = 10E-5

R = 50.
nx = 300
ny = 50

mesh = RectangleMesh(Point(0.,0.), Point(R, math.pi / 2.), nx, ny)

def solve_problem(lambd, R, count, mesh):
    
    class OuterBoundary(SubDomain):
        def inside(self, x, on_boundary):
            return on_boundary and near(x[0], R)

    class InnerBoundary(SubDomain):
        def inside(self, x, on_boundary):
            return on_boundary and x[0] <= l and x[1] < math.pi / 2. - tol

    Gamma_N = OuterBoundary()

    '''
    boundary_adaptor = EdgeFunction('bool', mesh)
    Gamma_N.mark(boundary_adaptor, True)
    for i in xrange(1):
        mesh = adapt(mesh, boundary_adaptor)
    '''

    boundary_parts = MeshFunction('size_t', mesh, mesh.topology().dim() - 1)
    

    Gamma_N = OuterBoundary()
    Gamma_N.mark(boundary_parts, 0)

    V = FunctionSpace(mesh, "Lagrange", 1)
    u = TrialFunction(V)
    v = TestFunction(V)

    bcs = [DirichletBC(V, Constant(1.), InnerBoundary()),
           DirichletBC(V, Constant(-0.94373464), OuterBoundary())]
           #DirichletBC(V, Expression('lambd * log(x[0] / p)', lambd=lambd, p=40.), OuterBoundary())]

    rho = Expression('x[0]')
    a = inner(grad(u), grad(v)) * rho * dx
    dS = ds(subdomain_data=boundary_parts)
    #L = Constant(lambd) * v * dS(0)
    L = Constant(0.) * v * rho *ds
    
    u = Function(V)

    problem = LinearVariationalProblem(a, L, u, bcs)
    solver = LinearVariationalSolver(problem)
    solver.solve()
    
    return u, mesh

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.tri as tri

def contour_plot(u, mesh, levels_count):
    triang = tri.Triangulation(*mesh.coordinates().reshape((-1, 2)).T,
                               triangles=mesh.cells())
    Z = u.compute_vertex_values(mesh)

    plt.figure()
    plt.tricontour(triang, Z, levels_count)
    plt.colorbar()
    plt.show()

In [2]:
def find_lambda(lambda_range, point, u_point, R, count, tol=10E-10):
    l, r = lambda_range
    lambd = (l + r) / 2
    u_point_numerical = solve_problem(lambd, R, count, mesh)[0](*point)
    while abs(u_point_numerical - u_point) > tol:
        if u_point_numerical - u_point > 0:
            r = lambd
        else:
            l = lambd
        lambd = (l + r) / 2
        print lambd
        u_point_numerical = solve_problem(lambd, R, count, mesh)[0](*point)
        print 'value:', u_point_numerical
    return lambd

In [ ]:
import numpy as np

def find_lambda(lambda_range, point, u_point, R, count, tol=10E-10):
    l, r = lambda_range
    lambda_range = np.linspace(l, r, 10)
    u_point_numerical = [solve_problem(lambd, R, count, mesh)[0](*point) for lambd in lambda_range]
    print u_point_numerical

In [3]:
#lambda_range = (-0.0046, -0.0042)
lambda_range = (-10., 10.)
lambd = find_lambda(lambda_range, (40., 0.), 0., 50., 100)
print 'lambda:', lambd

-5.0
value: -1.01675709994
-2.5
value: -0.484991532418
-1.25
value: -0.219108748656
-0.625
value: -0.0861673567741
-0.3125
value: -0.0196966608334
-0.15625
value: 0.013538687137
-0.234375
value: -0.0030789868482
-0.1953125
value: 0.00522985014439
-0.21484375
value: 0.0010754316481
-0.224609375
value: -0.00100177760005
-0.2197265625
value: 3.68270240218e-05
-0.22216796875
value: -0.000482475288015
-0.220947265625
value: -0.000222824131997
-0.220336914062
value: -9.29985539874e-05
-0.220031738281
value: -2.80857649827e-05
-0.219879150391
value: 4.37062951953e-06
-0.219955444336
value: -1.18575677316e-05
-0.219917297363
value: -3.74346910606e-06
-0.219898223877
value: 3.13580206767e-07
-0.21990776062
value: -1.71494444965e-06
-0.219902992249
value: -7.00682121461e-07
-0.219900608063
value: -1.93550957333e-07
-0.21989941597
value: 6.00146246925e-08
-0.219900012016
value: -6.67681663058e-08
-0.219899713993
value: -3.37677080031e-09
-0.219899564981
value: 2.83189269239e-08
-0.219899639487
va

In [18]:
#lambd = -0.219899695367
R = 50.
count = 100
u, mesh = solve_problem(lambd, R, count, mesh)

Calling FFC just-in-time (JIT) compiler, this may take some time.


In [19]:
plot(u, interactive=True)
contour_plot(u, mesh, 30)

NameError: name 'contour_plot' is not defined

In [14]:
import math
import cmath

class Exact_Solution(Expression):
    def eval(self, value, x):
        A = -1. / math.acosh(40. / l)
        B = 1.
        z = x[0] * cmath.rect(x[0], x[1])
        ln = math.log(abs( z / l + cmath.sqrt( (z / l) ** 2 - 1 ) ))
        value[0] = A * ln + B
    def value_shape(self):
        return (1,)

In [15]:
V = FunctionSpace(mesh, 'Lagrange', 3)
exact = interpolate(Exact_Solution(element=FiniteElement("Quadrature", triangle, 2)), V)

In [21]:
plot(exact, mesh, title='exact')
plot(u)
plot(u - exact, title='aaa')
interactive()

Found multiple domains, cannot return just one.


UFLException: Found multiple domains, cannot return just one.

In [20]:
print exact(R, 0.), exact(R, math.pi / 2.)
print u(R, 0.)

-0.943734632684 -0.943734650941
-0.94373464


In [ ]:
from dolfin import *
from mshr import *

w = 0.005
l = 1.
tol = 10E-14
R = 30.
count = 100
lambd = -0.2939453125

mesh = generate_mesh(Rectangle(Point(-R, -R), Point(R, R)) - Rectangle(Point(-l, -w), Point(l, w)), count)

class OuterBoundary(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary and (near(abs(x[0]), R) or near(abs(x[1]), R))

class InnerBoundary(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary and abs(x[0]) < l - tol and abs(x[1]) < w + tol

Gamma_N = InnerBoundary()

boundary_adaptor = EdgeFunction('bool', mesh)
Gamma_N.mark(boundary_adaptor, True)
for i in xrange(1):
    mesh = adapt(mesh, boundary_adaptor)

boundary_parts = MeshFunction('size_t', mesh, mesh.topology().dim() - 1)


Gamma_R = OuterBoundary()
Gamma_R.mark(boundary_parts, 0)

V = FunctionSpace(mesh, "Lagrange", 1)
u = TrialFunction(V)
v = TestFunction(V)

bcs = [DirichletBC(V, Constant(1.), InnerBoundary())]

a = inner(grad(u), grad(v)) * dx
#L = Constant(0.0) * v * ds
dS = ds(subdomain_data=boundary_parts)
L = Expression('lambd / sqrt(x[0]*x[0] + x[1]*x[1])', lambd=lambd) * v * dS(0)

u = Function(V)

problem = LinearVariationalProblem(a, L, u, bcs)
solver = LinearVariationalSolver(problem)
solver.solve()

plot(mesh)



import math
import cmath

class Exact_Solution(Expression):
    def eval(self, value, x):
        A = -1. / math.acosh(10. / l)
        B = 1.
        z = complex(x[0], x[1])
        ln = math.log(abs( z / l + cmath.sqrt( (z / l) ** 2 - 1 ) ))
        if x[0] < 0:
            ln *= -1.
        value[0] = A * ln + B
    def value_shape(self):
        return (1,)

Ve = FunctionSpace(mesh, 'Lagrange', 3)
exact = interpolate(Exact_Solution(element=FiniteElement("Quadrature", triangle, 2)), Ve)

plot(u - exact)

print u(10., 0.), exact(10., 0.)

interactive()

In [ ]:
from dolfin import *
from mshr import *

w = 0.005
l = 1.
tol = 10E-14
R = 30.
count = 100
lambd = -0.2939453125

mesh = generate_mesh(Circle(Point(0., 0.), R, 30) - Rectangle(Point(-R, -R), Point(0, R)) -
                     Rectangle(Point(0, -R), Point(R, 0)) - Rectangle(Point(0., -w), Point(l, w)), count)

class OuterBoundary(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary and (near(x[0], R) or near(x[1], R))

class InnerBoundary(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary and abs(x[0]) < l - tol and abs(x[1]) < w + tol

Gamma_N = InnerBoundary()

boundary_adaptor = EdgeFunction('bool', mesh)
Gamma_N.mark(boundary_adaptor, True)
for i in xrange(1):
    mesh = adapt(mesh, boundary_adaptor)

boundary_parts = MeshFunction('size_t', mesh, mesh.topology().dim() - 1)


Gamma_R = OuterBoundary()
Gamma_R.mark(boundary_parts, 0)

V = FunctionSpace(mesh, "Lagrange", 1)
u = TrialFunction(V)
v = TestFunction(V)

bcs = [DirichletBC(V, Constant(1.), InnerBoundary())]

a = inner(grad(u), grad(v)) * dx
dS = ds(subdomain_data=boundary_parts)
L = Expression('lambd / sqrt(x[0]*x[0] + x[1]*x[1])', lambd=lambd) * v * dS(0)

u = Function(V)

problem = LinearVariationalProblem(a, L, u, bcs)
solver = LinearVariationalSolver(problem)
solver.solve()

plot(mesh)



import math
import cmath

class Exact_Solution(Expression):
    def eval(self, value, x):
        A = -1. / math.acosh(10. / l)
        B = 1.
        z = complex(x[0], x[1])
        ln = math.log(abs( z / l + cmath.sqrt( (z / l) ** 2 - 1 ) ))
        if x[0] < 0:
            ln *= -1.
        value[0] = A * ln + B
    def value_shape(self):
        return (1,)

Ve = FunctionSpace(mesh, 'Lagrange', 3)
exact = interpolate(Exact_Solution(element=FiniteElement("Quadrature", triangle, 2)), Ve)

plot(u)
plot(exact, mesh)
plot(u - exact)

print u(10., 0.), exact(10., 0.)

interactive()